# Imports

In [90]:
import cv2
import numpy as np
import os
from multiprocessing import TimeoutError
from multiprocessing.pool import ThreadPool

# Funções

In [91]:
def timeout(seconds):
    def decorator(function):
        def wrapper(*args, **kwargs):
            pool = ThreadPool(processes=1)
            result = pool.apply_async(function, args=args, kwds=kwargs)
            try:
                return result.get(timeout=seconds)
            except TimeoutError as e:
                return e
        return wrapper
    return decorator

@timeout(60)
def read_user_name(pergunta):
    return input(pergunta)

def verifica_resposta_valida(pergunta):
    while(1):
        resp = read_user_name(pergunta)
        if isinstance(resp, TimeoutError):
            resp = 'n'
        
        if(resp != 'S' and resp != 's' and resp != 'N' and resp != 'n'):
            print('Resposta Invalida! Tente Novamente!!!!!')
        else:
            break
    
    return resp.upper()
    

def insere_nome(pergunta = 'Nome: '):
    resp = read_user_name('Nome: ')
    if isinstance(resp, TimeoutError):
            resp = 'n'
    
    return resp.lower()

def insere_senha(pergunta = 'Senha: '):
    resp = read_user_name(pergunta)
    if isinstance(resp, TimeoutError):
            resp = 'n'
    
    return(resp.lower())


def modo_administrador(users, camera, classificador, classificadorolho):
    user = str()
    password = str()
    print('########## Modo Administrativo ##########')

    while(1):
        user = insere_nome('Usuario: ')
        password = insere_senha()
        usuarios = []
        senhas = []
        for x in users:
            usuarios.append(x[0])
            senhas.append(x[1])
        
        if(user in usuarios):
            index = usuarios.index(user)
            if(password == senhas[index]):
                break
            else:
                print('Usuario ou senha incorretas!!!!!')
                resp = verifica_resposta_valida('Deseja tentar novamente? [S/N]')
                if(resp == 'N'):
                    return 1
        else:
            print('Usuario ou senha incorretas!!!!!')
            resp = verifica_resposta_valida('Deseja tentar novamente? [S/N]')
            if(resp == 'N'):
                return 1
    
    while(1):
        print('#### Acesso Concedido ####')
        print('1. Criar novo usuário')
        print('2. Apagar Usuario')
        print('3. Abrir Fechadura')
        print('4. Sair')
        q = input('Escolha uma das opcoes: ')

        if(q == '1'):
            while(1):
                user = insere_nome()
                password = insere_senha()
                confirmacao = insere_senha('Confirme a senha: ')
                if(password == confirmacao):
                    novo_usuario(user, password)
                    
                    users = ler_users()
                    usuarios = []
                    for x in users:
                        usuarios.append(x[0])
        
                    gravar_rosto(usuarios.index(user), camera, classificador, classificadorolho)
                    break
                else:
                    print('Senha e confirmacao nao batem!!!!!')
                    resp = verifica_resposta_valida('Deseja tentar novamente?[S/N] ')
                    if(resp == 'N'):
                        break

        elif(q == '2'):
            nome = insere_nome()
            confirmacao = verifica_resposta_valida('Voce deseja realmente apagar o rosto de ' + nome + '?[S/N] ')
            if(confirmacao == 'S'):
                apagar_rosto(usuarios.find(nome))
                
        elif(q == '3'):
            print('Porta aberta')
                
        else:
            break
    return 1

def modo_vigilancia(camera, classificador, classificadorolho):
    print('########## Modo Vigilância ##########')
    while(camera.isOpened()):
        conectado, imagem = camera.read() 
        if(conectado):
            imagemcinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
            facesdetectadas = classificador.detectMultiScale(imagemcinza, scaleFactor=1.5, minSize=(100,100))

        for (x,y,l,a) in facesdetectadas:
            cv2.rectangle(imagem, (x,y), (x+l, y+a), (0,0,255), 2)
        cv2.imshow('Face', imagem)

        #Botão para cancelar a execução
        key = cv2.waitKey(1) & 0xFF
        if (key==27):
            break
    camera.release()
    cv2.destroyAllWindows()


def ler_users():
    arq = open('users.txt', 'r')
    users = arq.readlines()
    senhas = []
    for i in range(len(users)):
        if(i < len(users)-1):
            users[i] = users[i][:-1]
            usuario = users[i][:users[i].find(',')]
            password = users[i][1+users[i].find(','):]
            
            lista = [usuario, password]
            senhas.append(lista)
            
        else:
            usuario = users[i][:users[i].find(',')]
            password = users[i][1+users[i].find(','):]
            
            lista = [usuario, password]
            senhas.append(lista)

    return senhas


def novo_usuario(novo_user, nova_pass):
    arq = open('users.txt', 'a')

    nova_linha = '\n' + novo_user + ',' + nova_pass
    arq.writelines(nova_linha)

    arq.close()
    print('Usuario ' + novo_user + ' adicionado!!!')
    return 1

def getimagemcomid(mostrar_ids = 0):
    caminhos = [os.path.join('fotos',f) for f in os.listdir('fotos')]
    faces=[]
    ids=[]
    for caminhoimagem in caminhos:
        imagemface = cv2.cvtColor(cv2.imread(caminhoimagem),cv2.COLOR_BGR2GRAY)
        id = int(os.path.split(caminhoimagem)[-1].split('.')[1])
        if(mostrar_ids == 1):
            print(id)
        ids.append(id)
        faces.append(imagemface)
    return np.array(ids), faces

def apagar_rosto(id_a_ser_removido):
    caminhos = [os.path.join('fotos',f) for f in os.listdir('fotos')]
    faces=[]
    ids=[]
    for caminhoimagem in caminhos:
        imagemface = cv2.cvtColor(cv2.imread(caminhoimagem),cv2.COLOR_BGR2GRAY)
        id = int(os.path.split(caminhoimagem)[-1].split('.')[1])
        if(id == id_a_ser_removido):
            os.remove(caminhoimagem)
        else:
            ids.append(id)
            faces.append(imagemface)
    
    eigenface = cv2.face.EigenFaceRecognizer_create()
    fisherface = cv2.face.FisherFaceRecognizer_create()
    lbph = cv2.face.LBPHFaceRecognizer_create()
                      
    ids, faces = getimagemcomid(1) 
    print('Apagando...')
    
    eigenface.train(faces, ids)
    eigenface.write('classificadorEigen.yml')
    
    fisherface.train(faces, ids)
    fisherface.write('classificadorFisher.yml')
    
    lbph.train(faces, ids)
    lbph.write('classificadorLBPH.yml')
    
    print(str(id) + ' Apagado!!!')

def gravar_rosto(id, camera, classificador, classificadorolho):
    amostra = 1
    numeroamostras = 25
    largura = 200
    altura = 200
    print('Capturando as faces...')
    while(camera.isOpened()):
        conectado, imagem = camera.read()  
        if(conectado):
            imagemcinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
            print(np.average(imagemcinza))
            facesdetectadas = classificador.detectMultiScale(imagemcinza, scaleFactor=1.5, minSize=(150,150))

        for (x,y,l,a) in facesdetectadas:
            cv2.rectangle(imagem, (x,y), (x+l, y+a), (0,0,255), 2)
            regiao = imagem[y:y + a, x:x + l]
            regiaocinzaolho = cv2.cvtColor(regiao, cv2.COLOR_BGR2GRAY)

            olhosdetectados = classificadorolho.detectMultiScale(regiaocinzaolho)

            for(ox, oy, ol, oa) in olhosdetectados:
                cv2.rectangle(regiao, (ox, oy), (ox+ol, oy+oa), (0, 255, 0), 2)

                if(cv2.waitKey(1) & 0xFF == ord('q')):
                    if(np.average(imagemcinza) > 70):
                        imagemface = cv2.resize(imagemcinza[y:y + a, x:x + l], (largura, altura))
                        cv2.imwrite('fotos/pessoa.' + str(id) + '.' + str(amostra) + '.jpg', imagemface)
                        print('Foto ' + str(amostra) + ' capturada com sucesso!')
                        amostra = amostra+1
                            
        cv2.imshow('Face', imagem)
        cv2.waitKey(1)
        if(cv2.waitKey(1) & 0xFF == 27):
            break
        if(amostra >= numeroamostras + 1):
            break
    print('Faces capturadas om sucesso')
    camera.release()
    cv2.destroyAllWindows() 

    eigenface = cv2.face.EigenFaceRecognizer_create()
    fisherface = cv2.face.FisherFaceRecognizer_create()
    lbph = cv2.face.LBPHFaceRecognizer_create()
                  
    ids, faces = getimagemcomid(1) 
    print('Treinando...')

    eigenface.train(faces, ids)
    eigenface.write('classificadorEigen.yml')

    fisherface.train(faces, ids)
    fisherface.write('classificadorFisher.yml')

    lbph.train(faces, ids)
    lbph.write('classificadorLBPH.yml')

    print('Treinamento Realizado')
    return 1

# Aplicação

In [92]:
classificador = cv2.CascadeClassifier("haarcascade-frontalface-default.xml")
classificadorolho = cv2.CascadeClassifier("haarcascade-eye.xml")
camera = cv2.VideoCapture(0)

users= ler_users()

resp = verifica_resposta_valida('Deseja entrar no modo administrador?[S/N] ')

if(resp == 'S'):
    modo_administrador(users, camera, classificador, classificadorolho)
    modo_vigilancia(camera, classificador, classificadorolho)
else:
    modo_vigilancia(camera, classificador, classificadorolho)


Deseja entrar no modo administrador?[S/N] s
########## Modo Administrativo ##########
Nome: victor
Senha: biral
#### Acesso Concedido ####
1. Criar novo usuário
2. Apagar Usuario
3. Abrir Fechadura
4. Sair
Escolha uma das opcoes: 1
Nome: tiago
Senha: 1234
Confirme a senha: 1234
Usuario tiago adicionado!!!
Capturando as faces...
96.291142578125
96.27514322916667
96.27514322916667
95.57657552083333
95.57657552083333
94.275791015625
94.275791015625
94.05387369791667
94.05387369791667
93.95676106770833
93.95676106770833
94.01441731770834
94.01441731770834
94.033349609375
94.033349609375
94.10701822916667
94.10701822916667
94.15723307291667
94.15723307291667
94.23593098958334
94.23593098958334
94.30677083333333
94.30677083333333
94.457744140625
94.457744140625
94.67981770833333
94.67981770833333
94.814169921875
94.814169921875
95.00227864583333
95.00227864583333
95.07745442708334
95.07745442708334
95.13278645833333
95.13278645833333
95.22943033854166
95.22943033854166
95.24161458333333
95.2